# Data analysis for the ISMAR article

In [2]:
# See: http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.formula.api import ols

from ast import literal_eval
from os import listdir
from os.path import join

Load the csv data files:

In [3]:
data_path = "data/"

participants_filepath = join(data_path, "participants.csv")
participants = pd.read_csv(participants_filepath)

trials = []
details = []
for file in listdir(data_path):
    filepath = join(data_path, file)
    if file.endswith("trials.csv"): # The trial files end with trials.csv
        trials.append(pd.read_csv(filepath))
    elif file.endswith("details.csv"):
        details.append(pd.read_csv(filepath))
trials = pd.concat(trials, ignore_index=True)
details = pd.concat(details, ignore_index=True)

In [19]:
projected_index = details.loc[details.projected_index_active == True][['participant_id', 'date_time', 'projected_index_position_x', 'projected_index_position_y']]